In [ ]:
# !pip install openai chromadb langchain

## 1. Load Energy Document Corpus
Use scraped or sample energy technical documents.

In [ ]:
with open('../../data/energy_text.txt') as f:
    docs = [line.strip() for line in f if line.strip()]
docs[:2]

## 2. Azure OpenAI Embeddings
Use text-embedding-ada-002 for vectorization.

In [ ]:
import openai
import os
openai.api_type = 'azure'
openai.api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai.api_key = os.getenv('AZURE_OPENAI_KEY')
def get_embedding(text):
    resp = openai.Embedding.create(input=text, engine='text-embedding-ada-002')
    return resp['data'][0]['embedding']

## 3. Store Embeddings in ChromaDB
Vector store for fast retrieval.

In [ ]:
import chromadb
client = chromadb.Client()
collection = client.create_collection('energy_docs')
for doc in docs:
    emb = get_embedding(doc)
    collection.add(documents=[doc], embeddings=[emb])

## 4. RAG Pipeline: Retrieval + Generation
Retrieve relevant docs and generate answer with Azure OpenAI.

In [ ]:
def rag_query(query):
    q_emb = get_embedding(query)
    results = collection.query(query_embeddings=[q_emb], n_results=3)
    context = '\n'.join([d for d in results['documents'][0]])
    prompt = f'Context: {context}\n\nQuestion: {query}\nAnswer:'
    resp = openai.ChatCompletion.create(
        engine='gpt-4',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return resp['choices'][0]['message']['content']
rag_query('How does reservoir pressure affect oil production?')

## 5. Cost Tracking
Log token usage and estimate cost.

In [ ]:
# Example: log token usage
def log_cost(response):
    usage = response['usage']
    tokens = usage['total_tokens']
    cost = tokens * 0.00002  # Example cost per token
    print(f'Tokens: {tokens}, Estimated Cost: ${cost:.4f}')

## 6. Prompt Engineering
Try few-shot and chain-of-thought prompts.

In [ ]:
few_shot_prompt = '''
Context: Oil production depends on reservoir pressure.

        Question: What is reservoir pressure?
        Answer: Reservoir pressure is the pressure of fluids within a reservoir.
        
        Question: How does it affect oil production?
        Answer: Higher reservoir pressure generally increases oil production.
        
        Question: What happens when pressure drops?
        Answer: Oil production typically declines.
        '''

## 7. Evaluation
Evaluate retrieval accuracy and answer quality.